 \# координаты каждой фигуры на доске представлена в виде tuple(x, y) где x, y - int от 0 до 7 включительно
 
 x - координата по оси x
 
 у - по y

## игра в шахматы 
(задания: 5, 6, 7 | суммарная сложность = 3)

#### 5	
Реализовать возможность «отката» ходов.

С помощью специальной команды можно возвращаться на ход (или заданное количество ходов) назад вплоть до начала партии. Информация о ходах в партии должна храниться в объектно-ориентированном виде.
 
    Сложность 1

#### 6	
Реализовать функцию подсказки выбора новой позиции фигуры: 

после выбора фигуры для хода функция визуально на поле показывает поля доступные для хода или фигуры соперника, доступные для взятия, выбранной фигурой. Информация о допустимых ходах должна храниться в объектно-ориентированном виде, алгоритм без модификации должен работать при добавлении новых типов фигур (задание берется совместно с Заданием 1).
 
    Сложность 1

#### 7	
Реализовать функцию подсказки угрожаемых фигур: 

она возвращает информацию о том, какие фигуры ходящего игрока сейчас находятся под боем (т.е. могут быть взяты соперником на следующий ход) и визуально выделяет их на поле. Функция отдельно указывает на наличие шаха королю. Информация о допустимых ходах должна храниться в объектно-ориентированном виде, алгоритм без модификации должен работать при добавлении новых типов фигур (задание берется совместно с Заданием 1).
 
    Сложность 1

используются файлы 
- constants.py
- pieces.py
- board.py 

In [ ]:
from board import Board
Board().play()

## игра в шашки

(задания: 2 | суммарная сложность = 2)

#### 2
На базе игры в шахматы реализовать игру в шашки. 

Разработать модификацию шахмат с минимальным вмешательством в существующий код.

сложность 2

In [1]:
from board import Board
from pieces import Piece
from constants import WHITE, BLACK

### класс Шашка

In [2]:
# одно подчеркивание в начале названия метода - protected method (доступен только внутри класса и в дочерних классах)
# двойное подчеркивание в начале названия метода - private method (доступен только внутри класса)
# это все на уровне соглашений, что взрослые люди не полезут туда, куда их просят не лезть

# класс Шашка
class Checker(Piece):
    def __init__(self, color, x, y):
        # как выглядит на доске
        self.image = ' H ' if color == WHITE else ' h '
        # цвет фигуры
        self.color = color
        # позиция на доске
        self.position = (x, y)

    def move(self, to_x, to_y, locations):
        # перемещение, смена координат
        moves_dict, flag1 = self.get_moves_dict(locations[0], locations[1])
        if moves_dict and (to_x, to_y) in list(moves_dict.keys()): # изменяем координаты фигуры на новые 
            self.position = (to_x, to_y)
            to_delete = moves_dict.get((to_x, to_y)) 
            flag2 = True
            if to_delete:  # удаляем
                if self.color == WHITE:
                    _, flag2 = self.get_moves_dict(locations[0], [tup for tup in locations[1] if tup != to_delete])
                else:
                    _, flag2 = self.get_moves_dict([tup for tup in locations[0] if tup != to_delete], locations[1])
            # переместить получилось, можем ли дальше продолжать делать ход, координаты съедаемой фигуры(если она есть)
            return True, flag1 and flag2, to_delete 
        return False, True, None
          
    def get_moves_dict(self, white_locations, black_locations):
        # получить список словарей положений, куда фигура может сходить и флаг - съедаем или нет фигуру
        moves_list = []
        killing_moves_list = {}
        if self.color == WHITE:
            targets = [(-1, -1), (1, -1)]
            enemies_list = black_locations
            friends_list = white_locations
        else:
            targets = [(1, 1), (-1, 1)]
            friends_list = black_locations
            enemies_list = white_locations
            
        # куда можно сходить, никого не съедая
        for target in targets:
            x, y = target
            if (self.position[0] + x,  self.position[1] + y) not in friends_list + enemies_list and \
                        0 <=  self.position[0] + x <= 7 and 0 <=  self.position[1] + y <= 7:
                    moves_list.append(( self.position[0] + x,  self.position[1] + y))
        # куда можно сходить, съедая
        targets = [(1, -1), (-1, -1), (1, 1), (-1, 1)]
        for target in targets:
            x, y = target
            if (self.position[0] + x,  self.position[1] +  y) in enemies_list and \
                    0 <= self.position[0] +  x <= 7 and 0 <=  self.position[1] + y <= 7 \
                    and ( self.position[0] + x * 2,  self.position[1] + y * 2) not in friends_list + enemies_list and \
                    0 <=  self.position[0] + x * 2 <= 7 and 0 <=  self.position[1] + y * 2 <= 7 :
                # если мы можем передвинуться на новую клетку
                killing_moves_list[(self.position[0] + x*2, self.position[1] + y*2)] = (self.position[0] + x,  self.position[1] + y) # добавляем ход в список ходов
        if killing_moves_list:
            # если есть возможность, то обязаны съесть, возвращаем только ходы, где съедается
            return killing_moves_list, True  # True - съедаем
        return {element: None for element in moves_list}, False  # False - не съедаем, преобразовываем к спискку словаря

    
    def get_moves_list(self, white_locations, black_locations):
        # получаем список позиций, куда фигура может переместиться на данном этапе в ходе
        moves_dict, flag = self.get_moves_dict(white_locations, black_locations)
        return list(moves_dict.keys())


### класс Дамка

In [3]:
class CheckerQueen(Checker):
    # наследуется о class Checker
    def __init__(self, color, x, y):
        super().__init__(color, x, y)
        # как выглядит на доске
        self.image = ' D ' if color == WHITE else ' d '

       
    def get_moves_dict(self, white_locations, black_locations):
        # получить список словарей положений, куда фигура может сходить и флаг - съедаем или нет фигуру
        moves_list = []
        killing_moves_list = {}
        if self.color == WHITE:
            enemies_list = black_locations
            friends_list = white_locations
        else:
            friends_list = black_locations
            enemies_list = white_locations

        targets =[(1, -1), (-1, -1), (1, 1), (-1, 1)]  # дамка ходит по диагонали в любом направлении
        # куда можно сходить, никого не съедая
        for target in targets:
            x, y = target
            for i in range(1,9):
                to_coords = None
                if (self.position[0] + x*i,  self.position[1] + y*i) not in friends_list + enemies_list and \
                            0 <=  self.position[0] + x*i <= 7 and 0 <=  self.position[1] + y*i <= 7:
                    # куда можно сходить не съедая пешку
                    to_coords = (self.position[0] + x*i,  self.position[1] + y*i)
                if to_coords:
                    moves_list.append(to_coords)
                    
        # куда можно сходить, съедая
        targets = [(1, -1), (-1, -1), (1, 1), (-1, 1)]
        for target in targets:
            x, y = target

            for i in range(1,9):
                if (self.position[0] + x*i,  self.position[1] + y*i) not in friends_list + enemies_list and \
                            0 <=  self.position[0] + x*i <= 7 and 0 <=  self.position[1] + y*i <= 7:
                    # если не дошли до пешки противника то перемещаемся на следующее поле
                    continue
                elif (self.position[0] + x*i,  self.position[1] +  y*i) in enemies_list and \
                        0 <= self.position[0] +  x*i <= 7 and 0 <=  self.position[1] + y*i <= 7 \
                        and ( self.position[0] + x*(i+1),  self.position[1] + y*(i+1)) not in friends_list + enemies_list and \
                        0 <=  self.position[0] + x*(i+1) <= 7 and 0 <=  self.position[1] + y*(i+1) <= 7 :
                    # если можем съесть
                    killing_moves_list[(self.position[0] + x*(i+1), self.position[1] + y*(i+1))] = (self.position[0] + x*i,  self.position[1] + y*i) # добавляем ход в список ходов
                    
                    for j in range(i+2, 9):
                        if (self.position[0] + x*j,  self.position[1] + y*j) not in friends_list + enemies_list and \
                            0 <=  self.position[0] + x*j <= 7 and 0 <=  self.position[1] + y*j <= 7:
                            killing_moves_list[(self.position[0] + x*j, self.position[1] + y*j)] = (self.position[0] + x*i,  self.position[1] + y*i) 
                else:
                    break
        if killing_moves_list:
            # если есть возможность, то обязаны съесть, возвращаем только ходы, где съедается
            return killing_moves_list, True  # True - съедаем
        return {element: None for element in moves_list}, False  # False - не съедаем, преобразовываем к спискку словаря


### класс Доска для шахмат

In [4]:
### - наследуется от Board
# класс Доска для шашек
class CheckersBoard(Board):
    def __init__(self):
        # номер хода
        self.count = 0
        # запись истории партии
        self.history = None # история для шашаек не реализуется
        # чей сейчас ход
        self.turn = WHITE  # WHITE или BLACK

        # белые фигуры     
        self.white_pieces_list = [
                                    Checker(WHITE, 0, 5),
                                    Checker(WHITE, 2, 5),
                                    Checker(WHITE, 4, 5),
                                    Checker(WHITE, 6, 5),
                                    Checker(WHITE, 1, 6),
                                    Checker(WHITE, 3, 6),
                                    Checker(WHITE, 5, 6),
                                    Checker(WHITE, 7, 6),
                                    Checker(WHITE, 0, 7),
                                    Checker(WHITE, 2, 7),
                                    Checker(WHITE, 4, 7),
                                    Checker(WHITE, 6, 7)
                                ]
        # черные фигуры 
        self.black_pieces_list = [
                                    
                                    Checker(BLACK, 1, 0),
                                    Checker(BLACK, 3, 0),
                                    Checker(BLACK, 5, 0),
                                    Checker(BLACK, 7, 0),
                                    Checker(BLACK, 0, 1),
                                    Checker(BLACK, 2, 1),
                                    Checker(BLACK, 4, 1),
                                    Checker(BLACK, 6, 1),
                                    Checker(BLACK, 1, 2),
                                    Checker(BLACK, 3, 2),
                                    Checker(BLACK, 5, 2),
                                    Checker(BLACK, 7, 2)
                                ]

### def get_piece(self, coords, color):  

### def get_white_locations(self):  

### def get_black_locations(self):

### def to_int_coords(self, xy: str):  

### def print_board(self, in_danger=False, coords=None): 

    def is_in_danger(self):
        # реализовать учитывая, что под опасностью не реализуется
        pass 
  
    def create_piece(self, color):
        # создаем и возвращаем фигуру не реализуется
        pass
    
    def moves_back(self, n_moves: int):
        # метод 'сделать ходы назад' не реализуется
        pass

    def get_killers(self):
        # список фигур, которые могут съесть фигуру противника в данном ходу
        lst = []
        if self.turn == WHITE:
            pieces_list = self.white_pieces_list
        else:
            pieces_list = self.black_pieces_list
            
        for p in pieces_list:
            moves_lst_dict, flag = p.get_moves_dict(self.get_white_locations(), self.get_black_locations())
            if flag:  # если у фигуры ходы с поеданием
                lst.append(p)
        return lst
            
    def move_piece(self, tmp_p):
        # сделать ход, кол-во ходов в партии +1
        # при необходимости обращает пешку в дамки
        # возвращает None

        not_end_move = True
        if self.turn == WHITE:  # если сейчас ход белых
            while not_end_move:
                # продолжаем перемещение пока можем двигаться те end_move != True
                self.print_board()
                to_coords = self.to_int_coords(input('куда переместить : '))
                if to_coords:
                    moved, not_end_move, killed_p_coords = tmp_p.move(to_coords[0], to_coords[1], (self.get_white_locations(), self.get_black_locations()))
                else:
                    moved = False
                    killed_p_coords = None
                    
                if not moved:  # если такой ход сделать было нельзя
                    print('\n ~ нельзя сделать такое перемещение! ~ ')
                    
                if killed_p_coords:  # если нужно, убираем съеденную фигуру с доски
                    bl_p = self.get_piece(killed_p_coords, BLACK) 
                    self.black_pieces_list.remove(bl_p)
                    
                if moved and tmp_p.position[1] == 0 and tmp_p.image != ' D ':
                    # превращаем в дамку
                    x = tmp_p.position[0]
                    self.white_pieces_list.remove(tmp_p)
                    tmp_p = CheckerQueen(WHITE, x, 0)
                    self.white_pieces_list.append(tmp_p)
                   # not_end_move = True
                
            self.turn = BLACK
            self.count += 1

        else:  # если сейчас ход черных (аналогично белым)
            while not_end_move:
                # продолжаем перемещение пока можем двигаться те end_move != True
                self.print_board()
                to_coords = self.to_int_coords(input('куда переместить : '))
                if to_coords:
                    moved, not_end_move, killed_p_coords = tmp_p.move(to_coords[0], to_coords[1], (self.get_white_locations(), self.get_black_locations()))
                else:
                    moved = False
                    killed_p_coords = None
                    
                if not moved:  # если такой ход сделать было нельзя
                    print('\n ~ нельзя сделать такое перемещение! ~ ')
                    
                if killed_p_coords:  # если нужно, убираем съеденную фигуру с доски
                    wh_p = self.get_piece(killed_p_coords, WHITE) 
                    self.white_pieces_list.remove(wh_p)
                    
                if moved and tmp_p.position[1] == 7 and tmp_p.image != ' d ':
                    # превращаем в дамку
                    x = tmp_p.position[0]
                    self.black_pieces_list.remove(tmp_p)
                    tmp_p = CheckerQueen(BLACK, x, 7)
                    self.black_pieces_list.append(tmp_p)
                    #not_end_move = True
            
            self.turn = WHITE
            self.count += 1
    
    def is_in_danger(self):
        # возвращает список координат фигур в опасности  не реализуется
        pass

    def play(self):  
        # игра (меню действий и общение с пользователем)
        self.print_board()
        print(f'-- {self.turn} --')
        menue = '0. закончить игру\n1. сделать ход\n? : '
        choice = input(menue)

        while choice != '0':
            from_coords = self.to_int_coords(input('выберите фигуру  : '))
            tmp_p = self.get_piece(from_coords, self.turn) 
            killers_list = self.get_killers()
            if tmp_p:
                if killers_list and tmp_p not in killers_list: 
                    print('\n ~ вы обязаны съесть фигуру противника! ~ ')
                else:
                    # если выбрали правильную фигуру 
                    self.move_piece(tmp_p)     
            else:
                print('\n ~ вы не можете выбрать фигуру по этим координатам! ~ ')
            self.print_board()
            
            if not self.black_pieces_list:
                print(f'----------------------------\n  белые выиграли !')
                return
            if not self.white_pieces_list:
                print(f'----------------------------\n  черные выиграли !')
                return
                
            print(f'-- {self.turn} --')
            print(f'-- ход № {self.count+1} --')
            choice = input(menue)

        # заканчиваем игру
        choice_2 = input('\n0) вы выиграли\n1) вы сдаетесь\n2) вы согласны на ничью\n? ')
        if choice_2 == '0':
            if self.turn != WHITE:
                print(f'----------------------------\n  черные выиграли !')
            else:
                print(f'----------------------------\n  белые выиграли !')
        elif choice_2 == '1':
            if self.turn != WHITE:
                print(f'----------------------------\n  белые выиграли !')
            else:
                print(f'----------------------------\n  черные выиграли !')
        else:  # если выбрали ничью или несуществующий вариант
            print(f'----------------------------\n  ничья !')


In [5]:
c = CheckersBoard()
c.play()


       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  -  h  -  h  -  h  -  h  |  8 
 7  |  h  -  h  -  h  -  h  -  |  7 
 6  |  -  h  -  h  -  h  -  h  |  6 
 5  |  -  -  -  -  -  -  -  -  |  5 
 4  |  -  -  -  -  -  -  -  -  |  4 
 3  |  H  -  H  -  H  -  H  -  |  3 
 2  |  -  H  -  H  -  H  -  H  |  2 
 1  |  H  -  H  -  H  -  H  -  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

-- white --
0. закончить игру
1. сделать ход
? : 1
выберите фигуру  : g3

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  -  h  -  h  -  h  -  h  |  8 
 7  |  h  -  h  -  h  -  h  -  |  7 
 6  |  -  h  -  h  -  h  -  h  |  6 
 5  |  -  -  -  -  -  -  -  -  |  5 
 4  |  -  -  -  -  -  -  -  -  |  4 
 3  |  H  -  H  -  H  -  H  -  |  3 
 2  |  -  H  -  H  -  H  -  H  |  2 
 1  |  H  -  H  -  H  -  H  -  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

куда переместить : h4

       A  B 

чтобы быстро проверить правильность игры, расставим фигуры определенным образом (создадим класс наследник CheckersBoard)

In [5]:
class TestBoard(CheckersBoard):
    def __init__(self):
        super().__init__()
        
        # белые фигуры     
        self.white_pieces_list = [
                                    Checker(WHITE, 3, 4),
                               
                                ]
        # черные фигуры 
        self.black_pieces_list = [
                                    
                                    Checker(BLACK, 2, 1),
                                    Checker(BLACK, 2, 3),
                                    Checker(BLACK, 5, 6),

                                ]

In [6]:
TestBoard().play()


       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  -  -  -  -  -  -  -  -  |  8 
 7  |  -  -  h  -  -  -  -  -  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  h  -  -  -  -  -  |  5 
 4  |  -  -  -  H  -  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  -  -  -  -  -  h  -  -  |  2 
 1  |  -  -  -  -  -  -  -  -  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

-- white --
0. закончить игру
1. сделать ход
? : 1
выберите фигуру  : d4

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  -  -  -  -  -  -  -  -  |  8 
 7  |  -  -  h  -  -  -  -  -  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  h  -  -  -  -  -  |  5 
 4  |  -  -  -  H  -  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  -  -  -  -  -  h  -  -  |  2 
 1  |  -  -  -  -  -  -  -  -  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

куда переместить : b6

       A  B 

## игра в шахматы с придуманными фигурами
(задания: 1 | суммарная сложность = 1)

#### 1
Придумать 3 новых вида фигур с оригинальными правилами перемещения и реализовать их классы. 

Создать модификацию шахмат с новыми фигурами с минимальным вмешательством в существующий код.

Сложность 1

используются файлы 
- constants.py 
- board.py (наследуем класс доска) 
- pieces.py (основной набор фигур)
- new_pieces.py (новые придуманные фигуры)
- board_with_new_p.py (доска с новыми фигурами)


In [1]:
from board_with_new_p import NewBoard
NewBoard().play()


       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  w  p  p  o  e  p  p  w  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  -  -  -  -  -  -  |  5 
 4  |  -  -  -  -  -  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  W  P  P  O  E  P  P  W  |  2 
 1  |  R  N  B  Q  K  B  N  R  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

-- white --
0. закончить игру
1. сделать ход
2. куда можно сходить фигурой
3. мои фигуры под ударом
4. откат хода/ходов
? : 1
-- ход № 1 --
выберите фигуру  : a2
куда переместить : 

 ~ неправильные координаты ! ~ 

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  w  p  p  o  e  p  p  w  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  -  -  -  -  -  -  |  5 
 4  |  -  -  -  -  -  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  W  P  P  O  E  P  P  W  |  2 
 1  |

0. закончить игру
1. сделать ход
2. куда можно сходить фигурой
3. мои фигуры под ударом
4. откат хода/ходов
? : 3

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  w  -  -  o  e  p  p  w  |  7 
 6  |  p  -  -  -  -  -  -  -  |  6 
 5  |  -  -  p  -  -  -  -  -  |  5 
 4  |  -  -  -  -  -  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  W  P  P  O  -  P  P  W  |  2 
 1  |  R  N  B  Q  K  B  N  R  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

-- white --
0. закончить игру
1. сделать ход
2. куда можно сходить фигурой
3. мои фигуры под ударом
4. откат хода/ходов
? : 1
-- ход № 5 --
выберите фигуру  : b2
куда переместить : b4

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  w  -  -  o  e  p  p  w  |  7 
 6  |  p  -  -  -  -  -  -  -  |  6 
 5  |  -  -  p  -  -  -  -  -  |  5 
 4  |  -  P  -  -  -  -  -  -  | 